# Data Exploration

### Imports


In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

<div class="alert alert-block alert-warning">
<b>Dask:</b> Multiprocessing package that can be used for (almost all) Pandas & Scikit-learn functionality.
 Ships with Anaconda, but not with MiniConda.
</div>

In [3]:
# from sklearn.grid_search import GridSearchCV
#from dklearn.grid_search import GridSearchCV   # voorbeeld van dklearn => dit gaan we zeker nodig hebben bij hyperopt etc.
# from dask.distributed import Client
# client = Client() # often http://localhost:8787/status

### Load Data In

In [ ]:
path_dominic = 'C:/Users/doist/OneDrive/Documenten/Business Analytics/Master/Year 1/Data Mining Techniques/Assignment 2/Data/'
path_titus = '2nd-assignment-dmt-2021/'

use_dominic = False
dir = path_dominic if use_dominic else path_titus

df = pd.read_csv(dir+'training_set_VU_DM.csv', parse_dates=['date_time'])
display(df)

In [ ]:
mem_size = df.memory_usage(index=True).sum() / 1_024**2

print(f"Size in Memory: {mem_size:.0f} MB")

In [ ]:
# Inspect inferred types
display(df.dtypes)

# Inspect numerical ranges
display(df.describe())
# print('Min. Values')
# print(df.min())
#
# print('Max. Values')
# print(df.max())

In [ ]:
# data types

# # memory + speed improvements
# optimal_dtypes = {'srch_id': np.uint32,
#                   'site_id': np.uint32,
#                   'visitor_location_country_id': np.uint8,
#                   'prop_country_id': np.uint8,
#                   'prop_id': np.uint32,
#                   'prop_starrating': np.uint8,
#                   'prop_brand_bool': np.uint8,
#                   'promotion_flag': (np.uint8,
#                   'srch_destination_id': np.uint16,
#                   'srch_length_of_stay': np.uint16,
#                   'srch_booking_window': np.uint16,
#                   'srch_adults_count': np.uint8,
#                   'srch_children_count': np.uint8,  # 255 kids max should suffice
#                   'srch_room_count': np.uint8,
#                   'srch_saturday_night_bool': np.uint8,
#                   'random_bool': np.uint8,
#                   'position': np.uint8,
#                   'click_bool': np.uint8,
#                   'booking_bool': np.uint8}
#
# for i in range(1, 9):
#     optimal_dtypes[f'comp{i}_rate'] = np.int8,
#     optimal_dtypes[f'comp{i}_inv'] = np.int8,     # assignment does not specify -1 but is exists??
#     # competitor_dtypes[f'comp{i}_rate_percent_diff'] = np.uint


# dfn = df.dtypes(optimal_dtypes)
# print(dfn.dtypes)


## Exploration

### Count of unique values per column:

In [ ]:
uniques = df.nunique()
display(uniques)

### % missing values per column

In [ ]:
missing_percent = 100 * df.isnull().sum() / df.shape[0]
display(missing_percent)

filled_columns = missing_percent[missing_percent == 0].index.tolist()
print("Non-missing columns:")
display(filled_columns)

### Searches per website

In [ ]:
groupby_website = df.groupby('site_id')['srch_id'].nunique() / df['srch_id'].nunique()

site_searches =  groupby_website.sort_values(ascending=False)
display(site_searches)
# site 5 == 'expedia.com'??

### Clicks/Books per position, grouped by random_bool

In [ ]:
# groupby_random_bool = df.groupby('random_bool')[['click_bool', 'booking_bool']].count()
groupby_position = df.groupby(['random_bool', 'position'])[['click_bool', 'booking_bool']].sum()
groupby_random_bool = groupby_position.groupby('random_bool').sum()
click_book_ratio = groupby_position / groupby_random_bool
display(groupby_position)
display(groupby_random_bool)
display(click_book_ratio)

### Location distributions

In [ ]:
groupby_home_country = df.groupby('visitor_location_country_id')['srch_id'].nunique()
distribution_home_country = 100 * (groupby_home_country / df['srch_id'].nunique()).sort_values(ascending=False)
display(distribution_home_country)
# Country 219 == USA??
# Country 100 == UK?? => Ursu (2015) suggests South-korea

groupby_prop_country = df.groupby('prop_country_id')['srch_id'].nunique()
distribution_prop_country = 100 * (groupby_prop_country / df['srch_id'].nunique()).sort_values(ascending=False)
display(distribution_prop_country)

groupby_destination_id = df.groupby('srch_destination_id')['srch_id'].nunique()
distribution_destination_id = 100 * (groupby_destination_id / df['srch_id'].nunique()).sort_values(ascending=False)
display(distribution_destination_id)

# 8192 == NYC??
# 4562 == LA?? Miami??

### Drop Columns

In [ ]:
competitor_cols = []
for i in range(1, 9):
    competitor_cols += [f'comp{i}_rate', f'comp{i}_inv', f'comp{i}_rate_percent_diff']
drop_cols = competitor_cols + ['gross_bookings_usd', 'position']

df_stage1 = df.drop(columns=drop_cols)

### More Drops:

In [ ]:
# left-over with missings
left_overs = [col
              for col in df_stage1.columns
              if not col in filled_columns]
print("Left-over columns for drop/imputation")
display(missing_percent[left_overs])
drop_cols_extra = ['visitor_hist_starrating', 'visitor_hist_adr_usd', 'srch_query_affinity_score']  # voor nu even droppen
df_stage1.drop(columns=drop_cols_extra, inplace=True)


## Fill Na
### TODO: prop_review_score per DMT 2020 group 95
#### prop_review_score & prop_location_score2 imputed per country, lower quantile as baseline => cited by VU boys and Liu et al

In [ ]:
prop_score = df_stage1.groupby('prop_id')['prop_review_score'].max()
no_reviews = prop_score[prop_score == 0].index
display(no_reviews)
print(f"% properties with no reviews {100 * len(no_reviews) / len(prop_score)}")

# lots of properties with no reviews ever
# option 1: keep imputing like papers do
# option 2: use backfill if reviews show up later and then impute like papers do
# temporarily drop prop_review_score until decision made
df_stage1.drop(columns=['prop_review_score'], inplace=True)

In [ ]:
# grouped by 'srch_destination_id' (city) would be more precise, but still has some NaN's
country_score2 = df_stage1.groupby('prop_country_id')['prop_location_score2'].quantile(0.25)

score2_country_join = pd.merge(df_stage1, country_score2, on='prop_country_id', how='left')['prop_location_score2_y']
print(f"Still {score2_country_join.isnull().sum()} unsolved NaN's")

# Fill 2 left-overs
score2_country_join[score2_country_join.isnull()] = country_score2.mean()
print(f"Still {score2_country_join.isnull().sum()} unsolved NaN's")

df_stage2 = df_stage1.copy()
nan_mask = df_stage2['prop_location_score2'].isnull()
display(nan_mask)
df_stage2.loc[nan_mask, 'prop_location_score2'] = score2_country_join[nan_mask]

In [ ]:
# TODO: Impute distance matrix using shortest path
# drop for now
df_stage2.drop(columns=['orig_destination_distance'], inplace=True)


In [ ]:
# Group 95 does not give an alpha => this needs reasoning
alpha = 0.01
price_usd = df_stage2['price_usd']
price_usd_winsorized = price_usd.clip(lower=price_usd.quantile(alpha), upper=price_usd.quantile(1-alpha))

display(price_usd_winsorized)
outliers = price_usd[price_usd != price_usd_winsorized]
display(outliers)

df_stage3 = df_stage2.copy()
df_stage3['price_usd'] = price_usd_winsorized

## Temporary output

In [ ]:
missing_percent = 100 * df_stage3.isnull().sum() / df_stage3.shape[0]
print("% missing:")
display(missing_percent)

df_stage3.to_pickle('df_temporary.pkl')

# Feature engineering:










Check class inbalance:

In [ ]:
df.groupby('click_bool')['srch_id'].count()

In [ ]:
df.groupby('booking_bool')['srch_id'].count()

In [ ]:
print('Total instances not clicked and not booked = ', df.groupby('click_bool')['srch_id'].count()[0])
print('Total instances clicked but not booked = ', (df.groupby('click_bool')['srch_id'].count()[1]-df.groupby('booking_bool')['srch_id'].count()[1]))
print('Total instances clicked and booked = ', df.groupby('booking_bool')['srch_id'].count()[1])

In [ ]:
#Check if it is true that all instances booked are also clicked:
df[df['booking_bool']==1]['click_bool'].describe()

In [ ]:
def boxplot_compare(column):
    not_clicked_not_booked = df.loc[(df.click_bool==0) & (df.booking_bool==0)][column]
    clicked_not_booked = df.loc[(df.click_bool==1) & (df.booking_bool==0)][column]
    clicked_and_booked = df.loc[df.booking_bool==1][column]
    
    plt.figure(figsize=(16,8))
    sns.boxplot(data=[not_clicked_not_booked, clicked_not_booked, clicked_and_booked])
    plt.xticks(plt.xticks()[0], ['Not clicked or booked', 'Clicked but not booked', 'Booked'])

In [ ]:
boxplot_compare('position')

In [ ]:
boxplot_compare('prop_starrating')

In [ ]:
boxplot_compare('prop_review_score')

In [ ]:
boxplot_compare('prop_location_score1')

In [ ]:
df.groupby('booking_bool')['promotion_flag'].mean()

In [ ]:
df.groupby('click_bool')['promotion_flag'].mean()

Check if there is a correlation between variables and position:

In [ ]:
df_non_random = df[df['random_bool']==0]

In [ ]:
sns.boxplot(x='prop_review_score', y='position', data=df_non_random)

In [ ]:
sns.boxplot(x='prop_starrating', y='position', data=df_non_random)

### Bookings seasonality

In [ ]:
df_stage3['checkin_date'] = df_stage3['date_time'] + pd.to_timedelta(df_stage3['srch_booking_window'], unit='D')  # speedup with vectorize?,
display(df_stage3[['date_time', 'checkin_date']])
df_stage3['checkin_weeknr'] = df_stage3['checkin_date'].dt.isocalendar().week

groupby_weeknr = df_stage3.groupby('checkin_weeknr')['srch_id'].nunique()
distr_weeknr = groupby_weeknr / df_stage3['srch_id'].nunique()
display(groupby_weeknr)

import matplotlib.pyplot as plt
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
x = distr_weeknr.index.tolist()
y = distr_weeknr.values

ax.bar(x, y, 0.1, color='r')
ax.set_ylabel('Booking Ratio')
ax.set_xlabel('Week Nr.')
ax.set_title('Bookings Seasonality')
ax.set_title('Bookings Seasonality')
plt.show()

### Date Features

In [ ]:
def mine_date_features(dt_series, prefix=''):
        df = pd.DataFrame()
        df['weekday'] = dt_series.dt.weekday
        df['monthday'] = dt_series.dt.day
        df['month'] = dt_series.dt.month
        df['week'] = dt_series.dt.week
        df['year'] = dt_series.dt.year
    
        df.columns = list(map(lambda s: prefix + s, df.columns.tolist()))
        return df

In [ ]:
df_stage3['checkout_date'] = df_stage3['checkin_date'] + pd.to_timedelta(df_stage3['srch_length_of_stay'], unit='D')

df_srchdate_features = mine_date_features(df_stage3['date_time'], prefix='srchdate_')
df_srchdate_features['srchdate_hour'] = df_stage3['date_time'].dt.hour

df_checkin_features = mine_date_features(df_stage3['checkin_date'], prefix='checkin_')
df_checkout_features = mine_date_features(df_stage3['checkout_date'], prefix='checkout_')

df_stage4 = pd.concat([df_stage3, df_srchdate_features, df_checkin_features, df_checkout_features], axis=1)
display(df_srchdate_features)
display(df_checkin_features)
display(df_checkout_features)

### Count # of weekdays contained in booking_window (i.e. Sun: 1, Sat: 1, mon: 1, tue: 0, wed: 0 ...)

In [ ]:
import calendar
def dayofweek_count(dt_start, dt_end):
    dt_range = pd.date_range(dt_start, dt_end, freq='d')

    cnt = {day: 0 for day in calendar.day_name}
    for date in dt_range:
        cnt[date.day_name()] += 1
    return pd.Series(cnt)


srch_grouped = df_stage4.groupby('srch_id').first()
srch_window = pd.DataFrame(srch_grouped[['checkin_date', 'checkout_date']])
display(srch_window)

In [ ]:
srch_daycounts = srch_window.apply(lambda df: dayofweek_count(df['checkin_date'], df['checkout_date']), axis=1)

display(srch_daycounts)

In [ ]:
df_stage5 = pd.merge(df_stage4, srch_daycounts, on='srch_id', how='left')
display(df_stage5.head())

In [ ]:
# Check if join is correct
print(df_stage4.shape)
print(df_stage5.shape)
missings = df_stage5[list(calendar.day_name)].isnull().sum() / df_stage5.shape[0]
print(missings)


### Holiday Features (if contained in booking window)

In [ ]:
import holidays
holidays = holidays.UnitedStates(years=[2012, 2013, 2014])  # easter is missing but fuck that

In [ ]:
from collections import defaultdict
def booking_contains_holidays(dt_start, dt_end, holidays):
    dt_range = [dt.date() for dt in pd.date_range(dt_start, dt_end, freq='D')]

    contained = defaultdict(bool)
    for dt_holiday, name_holiday in holidays.items():
        if contained[name_holiday]: # prevents override of holiday from another year
            continue

        contained[name_holiday] = dt_holiday in dt_range
    return pd.Series(contained)

In [ ]:
srch_holidays = srch_window.apply(lambda df: booking_contains_holidays(df['checkin_date'], df['checkout_date'], holidays), axis=1)

In [ ]:
nr_of_holidays = (srch_holidays == True).sum()
print(nr_of_holidays)

In [ ]:
df_stage6 = pd.merge(df_stage5, srch_holidays, on='srch_id', how='left')

### Pricing features

In [ ]:
df_prop = df[['prop_id', 'srch_id', 'date_time', 'price_usd']].set_index(['prop_id', 'date_time']).sort_index()

df_prop['last_price'] = df_prop.groupby(level=0)['price_usd'].shift().fillna(df_prop['price_usd'])  # use same price
df_prop['diff_last_price'] = (df_prop['price_usd'] - df_prop['last_price']) / df_prop['last_price']

In [ ]:
def rolling_min(series):
    expand_min = series.expanding().min()

    print(series)

    
df_prop['min_price'] = df_prop[:100].groupby(level=0)['price_usd'].apply(rolling_min)
display(df_prop.head(65))




In [ ]:
# TODO: add price differences per property (from last historic, mean and max)
# TODO: price difference whole dataset => Ursu (2015) suggests cheaper properties have more clicks


In [ ]:
# TODO: plot search impressions compared to time of day -> compare USA (219) vs rest to check timezone

In [ ]:
# TODO: prop_review_score by average of hotel?